# Trabalho 2

## 4.1

### Imports

In [47]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

### Variáveis

In [ ]:
ficheiro = "../../ficheiros/Trabalho 2/Dados_Trabalho_TP2.csv"
colunas_classes_binarias = ['Genero', 'Historico_obesidade_familiar', 'FCCAC', 'Fumador', 'MCC']  # Colunas de valores binários
colunas_classes_dummies = ['CCER', 'CBA', 'TRANS']


# Funções auxiliares

In [ ]:
def titulo(texto: str):
    print(f"\033[21;30;44m{texto}\033[0m")


def etiqueta_e_valor(etiqueta: str, valor: str = ""):
    print(f"\033[0;94m{etiqueta}: \033[1;94m{valor}\033[0m")


def texto(texto: str, cor="94"):
    print(f"\033[0;{cor}m{texto}\033[0m")


def undersplit(texto):
    return " ".join(texto.split("_"))


def histograma(
        dados: pd.Series,
        grupos: int = 20,
        kde: bool = True,
        size_x: int = 4,
        size_y: int = 4,
        titulo: str = "",
        etiqueta_x: str = "",
        etiqueta_y: str = "",
        color: str = "skyblue",
        edgecolor: str = "black",
):
    """Gera um histograma de forma isolada.
    
    A dimensão final do gráfico é controlada pela relação entre os valores size_x e size_y.
    """
    hist_fig, hist_axes = plt.subplots(figsize=(size_x, size_y))
    sns.histplot(dados, bins=grupos, kde=kde, color=color, edgecolor=edgecolor)
    hist_axes.set_title(titulo)
    hist_axes.set_xlabel(etiqueta_x)
    hist_axes.set_ylabel(etiqueta_y)
    plt.tight_layout()


def tarte(
        dados: pd.Series,
        size_x: int = 4,
        size_y: int = 4,
        autopct="%.2f%%",
        titulo: str = "",
        **kwargs,
):
    pie_fig, pie_axes = plt.subplots(figsize=(size_x, size_y))
    pie_axes.pie(
        dados,
        autopct=autopct,
        **kwargs
    )
    pie_fig.suptitle(titulo)
    return pie_fig, pie_axes



### Inicializações

In [ ]:
warnings.filterwarnings("ignore")  # Desabilitar warnbings.
plt.style.use(
    "style/estilo.mplstyle")  # Garantir que se utiliza um estilo definido centralmente e comum a todos os gráficos.
label_encoder = LabelEncoder()

### 4.1.1

#### Leitura de dados

Como os dados lidos não são em grande quantidade, vamos criar uma cópia do **dataset** para trabalharmos nesta, para poder ter acesso aos "originais" sem ter de os ler novamente do ficheiro.
Por exemplo em alguns dos gráficos, a utilização das *classes* (valores originais) como *etiqueta* é mais elucidativa que usar os valores codificados.  

Na análise ao **dataset** verifica-se que a primeira coluna, identificada como "Unnamed" tem o indíce de cada registo de dados, algo que nos é fornecido pelo **pandas**, e portanto podemos eliminar esta coluna. 
Esta operação estaria incluída nas tarefas do ponto **4.1.4**, mas realizar neste ponto facilita a criação dos gráficos no próximo ponto.

In [ ]:
dados_lidos = pd.read_csv(ficheiro)
dados_trabalho = dados_lidos.copy()
dados_trabalho = dados_trabalho.drop(columns=dados_lidos.columns[0], axis=1)

### Dimensão dos dados

In [ ]:
nr_linhas, nr_colunas = dados_trabalho.shape
titulo("Dimensão dos dados")
etiqueta_e_valor("Número de linhas:", nr_linhas)
etiqueta_e_valor("Número de colunas:", nr_colunas)

### Análise do dataset

#### Atributos

Descrição dos atributos, segundo a documentação. 

| Sigla | Descrição                                                         |
|-------|-------------------------------------------------------------------|
| FCCAC | Frequência de Consumo de Comida Altamente Calórica                | 
| FCV   | Frequência de Consumo de Vegetais                                 | 
| NRP   | Número de Refeições Principais                                    |
| CCER  | Consumo de Comida Entre Refeições                                 |
| CA    | Consumo de Água                                                   |
| CBA   | Consumo de Bebidas Alcoólicas                                     |
| MCC   | Monitorização do Consumo Calorias Histórico de Obesidade Familiar | 

#### Informação sobre o dataset

A função ``info()`` apresenta a informação das colunas (atríbutos): Index da coluna, nome (se tiver, quantidade de registos *não nulos* e o tipo de dados de cada um deles.

In [ ]:
dados_trabalho.info()

#### Amostra do dataset

Para uma rápida visualização dos dados, temos as funções ``head(n)`` e ``tail(n)`` que nos dão *n* linhas, por defeito 5, respetivamente do início ou fim do **dataset**.

In [ ]:
dados_trabalho.head()

In [ ]:
dados_trabalho.tail()

#### Descrição do dados do dateset

A função ``describe()`` analisa os dados do **dataset** e retorna um conjunto de estatísticas descritivas.

In [ ]:
dados_trabalho.describe(include="all")

#### Identificar as classes da coluna Label

In [ ]:
titulo("Classes da coluna \033[1mLabel\033[0m")
for classe in dados_trabalho['Label'].unique():
    texto(classe)

### Identificar as classes da coluna CCER

In [ ]:
titulo("Classes da coluna \033[1mCCER\033[0m")
for classe in dados_trabalho['CCER'].unique():
    texto(classe)

#### identificar classes da coluna TRANS

In [ ]:
titulo("Classes da coluna \033[1mTRANS\033[0m")
for classe in dados_trabalho['TRANS'].unique():
    texto(classe)


### 4.1.2



#### Derivar atributo (IMC)

Depois de criar o novo atributo, podemos usar qualquer uma das funções que foram utilizadas para analisar o **dataset**, neste caso tratando-se de valores numéricos consideramos que a que a mais útil é a ``describe()``.

Podemos também remover as colunas utilizadas para este cálculo uma vez que não vão ser necessárias.

Faz sentido alguma limpeza do **Dataset** neste momento, pois vai facilitar o nosso trabalho. 

In [ ]:
dados_trabalho['IMC'] = dados_trabalho['Peso'] / (dados_trabalho['Altura'] * dados_trabalho['Altura'])
dados_trabalho.drop(columns=['Peso', 'Altura'], inplace=True)


#### Rever o **dataset** após as alterações

In [ ]:
dados_trabalho.describe(include="all")

Finalmente, usamos novamente a função ``info()`` para confirmar as *series* com que vamos trabalhar de seguida. 

In [ ]:
dados_trabalho.info()

### 4.1.3 Analisar os atributos do conjunto de dados mais significativos, usando gráficos, ...


#### Histogramas

Com estes gráficos podemos ver a distribuição dos valores por "segmentos" de valores.

In [ ]:
sns.set(style="whitegrid")
hist_columns_plot = ['FCV', 'NRP', 'CA', 'FAF', 'TUDE', 'IMC']

for idx, etiqueta_serie in enumerate(hist_columns_plot):
    histograma(
        dados=dados_trabalho[etiqueta_serie],
        grupos=20,
        kde=True,
        size_x=10,
        size_y=4,
        titulo=f"Distribuição de {etiqueta_serie}",
        etiqueta_x=etiqueta_serie,
        etiqueta_y="Frequência",
        color="skyblue",
        edgecolor="black",
    )


#### Pie Charts

A função ``tarte()`` retorna a ``Fig`` e o ``Axes`` para se poder personalizar mais o gráfico.
É também possível passar parâmetros adicionais (*kwargs*), os quais serão usados na invocação de ``pie()``. 

In [ ]:
tarte(
    dados=dados_trabalho["Historico_obesidade_familiar"].value_counts(),
    size_x=4,
    size_y=4,
    titulo="Histórico obesidade familiar",
    labels=dados_lidos["Historico_obesidade_familiar"].unique(),
)

tarte(
    dados=dados_trabalho["FCCAC"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Frequência de Consumo de Comida Altamente Calórica",
    labels=dados_lidos["FCCAC"].unique(),
)

tarte(
    dados=dados_trabalho["Fumador"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Fumador",
    labels=dados_lidos["Fumador"].unique(),
)

tarte(
    dados=dados_trabalho["MCC"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Monitorização do Consumo Calorias",
    labels=dados_lidos["MCC"].unique(),
)

tarte(
    dados=dados_trabalho["CBA"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Consumo de Bebidas Alcoólicas",
    labels=dados_lidos["CBA"].unique(),
)

tarte(
    dados=dados_trabalho["TRANS"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Transporte",
    labels=[undersplit(texto) for texto in dados_lidos["TRANS"].unique()],
)

tarte(
    dados=dados_trabalho["Label"].value_counts(),
    size_x=5,
    size_y=5,
    titulo="Classificação do peso",
    labels=[undersplit(texto) for texto in dados_lidos["Label"].unique()],
)

#### visualização de outliers

In [ ]:
boxplots_colunas = ["IMC", "Idade"]

for coluna in boxplots_colunas:
    if (dados_trabalho[coluna].dtype != 'object'):
        dados_trabalho.boxplot(
            column=[coluna],
            by="Label",
            grid=False,
            figsize=(8, 6),
            return_type="dict",
            rot=90,
        )
        plt.title(coluna)

plt.show()

### 4.1.4

#### a) Identificação de NAN e "limpar" o dataset.

Verificar se existem valores nulos.

In [ ]:
dados_trabalho[dados_trabalho.isnull().any(axis=1)]

Verificar se existem valores NaN

In [ ]:
dados_trabalho[dados_trabalho.isna().any(axis=1)]

#### Transformar classes em valores

No mesmo passo em que fazemos a conversão, mostramos os valores finais.
Vamos fazer esta operação para as colunas em que apenas temos 2 classes, para as restantes usamos o ``get_dummies``:

genero, historico_obesidade_familiar, FCCAC, 

In [ ]:
for coluna in colunas_classes_binarias:
    dados_trabalho[coluna] = label_encoder.fit_transform(dados_trabalho[coluna].values)
    etiqueta_e_valor(coluna, dados_trabalho[coluna].unique())

#### Notas sobre o pré-processamento dos dados

Não foram encontrados valores nulos, nem valor **NaN**, não sendo por isso necessário fazer tratamento adicionais ao dados neste contexto. 

Algum do trabalho de limpeza foi realizado do ponto 4.1.2. 

TODO: Comentário sobre dados inconsistentes e outliers
TODO: Selecionar atributos (?)
TODO: Normalização 

#### Normalização

A normalização consiste em transformar o intervalo de valores de um atributo em valores compreendidos entre 0 e 1, para eliminar os efeitos da escala utilizada.

Podemos utilizar o ``MinMaxScaler``. Apenas as colunas de valores numéricos devem ser normalizadas:

Idade, FCV, NRP, CA, FAF, TUDE, IMC
 

In [ ]:
colunas_numericas = ["Idade", "FCV", "NRP", "CA", "FAF", "TUDE", "IMC"]
scaler = MinMaxScaler()
dados_trabalho.loc[:, colunas_numericas] = scaler.fit_transform(dados_trabalho.loc[:, colunas_numericas])
dados_trabalho

#### Gerar Dummies para as colunas de classes



CCER, CBA e TRANS

In [37]:
dados_trabalho = pd.get_dummies(dados_trabalho, dtype=float)
dados_trabalho

,Genero,Idade,Historico_obesidade_familiar,FCCAC,FCV,NRP,CCER,Fumador,CA,MCC,...,IMC,Label_Excesso_Peso_Grau_I,Label_Excesso_Peso_Grau_II,Label_Magreza_Grau_I,Label_Magreza_Grau_II,Label_Magreza_Grau_III,Label_Obesidade_Moderada,Label_Obesidade_Mórbida,Label_Obesidade_Severa,Label_Peso_Normal
0,0,0.148936,1,0,0.5,0.666667,2,0,0.500000,0,...,0.301162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0.148936,1,0,1.0,0.666667,2,1,1.000000,1,...,0.297240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,0.191489,1,0,0.5,0.666667,2,0,0.500000,0,...,0.284736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,0.276596,0,0,1.0,0.666667,2,0,0.500000,0,...,0.366359,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.170213,0,0,0.5,0.000000,2,0,0.500000,0,...,0.405778,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,0,0.148443,1,1,1.0,0.666667,2,0,0.364070,0,...,0.843697,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2107,0,0.169850,1,1,1.0,0.666667,2,0,0.502565,0,...,0.813032,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2108,0,0.181362,1,1,1.0,0.666667,2,0,0.527097,0,...,0.807793,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2109,0,0.220467,1,1,1.0,0.666667,2,0,0.926170,0,...,0.821749,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### 4.1.5 Matriz de Correlação

In [ ]:
dados_corr = dados_trabalho.copy()
fig_corr = plt.figure(figsize=(12, 8))
etiquetas_corr = [undersplit(texto) for texto in dados_corr.columns]

for coluna in dados_trabalho.columns:
    dados_corr[coluna] = label_encoder.fit_transform(dados_corr[coluna].values)

sns.heatmap(dados_corr.corr(method="spearman").abs(), cmap='coolwarm', annot=True, fmt='.2f', mask=False)

plt.xticks(ticks=[i + 0.5 for i in range(len(dados_corr.columns))], labels=etiquetas_corr, rotation=45, ha='right')
plt.yticks(ticks=[i + 0.5 for i in range(len(dados_corr.columns))], labels=etiquetas_corr)
plt.title('Matriz de Correlação')
plt.tight_layout()

# plt.show()

Por este diagrama podemos ver que não existe uma forte correlação entre nenhum dos atributos. 

### 4.1.6

#### Hold-out

**Hold-out** é o processo de dividir um conjunto de dados em 2 sub-conjuntos de forma aleatória, um de maior dimensão que será utilizado para treinar um algoritmo de Machine Learning e o outra para o testar. 

Predictor: Idade
Target: IMC


TODO: Utilizar os dados que o Pedro enviou e Trabalho 1 como suporte. Mas fazer o modelo sobre 80% dos valores e usar os outros 20% para o cálculo ddo MAE e RMSE

Para o ponto d)



In [ ]:
y = dados_trabalho.IMC
X = dados_trabalho[["Idade"]].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=100)


#### Apresentar informação sobre os datasets de treino e aprendizagem

In [ ]:
titulo("|\t\t\tDimensão\t\t\t|")
titulo("|\tTreino\t\t|\tTeste\t\t|")
texto(f"|\t{X_train.shape}\t|\t{X_test.shape}\t|")
texto(f"|\t{y_train.shape}\t\t|\t{y_test.shape}\t\t|")


#### Inicialização do algoritmo

In [ ]:
regr_lin = LinearRegression()

#### Ajustar os dados

In [ ]:
regr_lin.fit(X_train, y_train)

#### 4.1.6 a)

In [ ]:
etiqueta_e_valor("Equação da reta:", f" y = {regr_lin.intercept_:.3f} + {regr_lin.coef_[0]:.3f} x")

#### 4.1.6 b)

In [ ]:
sns.regplot(x="Idade", y="IMC", data=dados_trabalho, order=1, ci=None, scatter_kws={'color': 'r', 's': 9})
plt.xlim(10, 65)
plt.ylim(ymin=0);

Aplicar o modelo sobre o conjunto de teste.

In [ ]:
y_pred = regr_lin.predict(X_test)


#### MAE

$$MAE = \sum_{i=1}^{D}|x_i-y_i|$$

In [ ]:
MAE = np.mean(abs(y_test - y_pred))
etiqueta_e_valor("MAE no conjunto de dados de teste:", f"{MAE:.3f}")

#### RMSE

$$RMSE = \sqrt{\frac{\sum_{i=0}^{N - 1} (y_i - \hat{y}_i)^2}{N}}$$

In [ ]:
RMSE = np.sqrt(np.mean((y_test - y_pred) ** 2))
etiqueta_e_valor("RMSE no conjunto de dados de teste:", f"{RMSE:.3f}")

#### Repetir o cálculo mas com outros previsores

O preditor vai mudando, podemos ter uma lista com o nome das colunas e fazer um ciclo que realiza os cálculos e guardamos o resultado num dicionário

In [39]:
# O target não vai mudar.
y = dados_trabalho.IMC

# O preditor vai mudando, podemos ter uma lista com o nome das colunas
lista_preditores = ['Genero', 'Historico_obesidade_familiar', 'FCCAC', 'FCV', 'NRP', 'CCER', 'Fumador', 'CA',
                    'MCC', 'FAF', 'TUDE', 'CBA', 'TRANS']
# Falta o Label


# Resultados
resultados = {}

for preditor in lista_preditores:
    X_preditor = dados_trabalho[[preditor]].to_numpy()
    X_train_1, X_test_1, y_train, y_test = train_test_split(X_preditor, y, train_size=0.8, test_size=0.2,
                                                            random_state=100)
    line_regr = LinearRegression()
    modelo = line_regr.fit(X_train_1, y_train)

    y_pred_1 = line_regr.predict(X_test_1)

    resultados[preditor] = {
        # "X_train": X_train_1,
        # "X_test": X_test_1,
        # "y_train": y_train,
        # "y_test": y_test,
        "modelo": modelo,
        "y_pred": y_pred_1,
        "MAE": np.mean(abs(y_test - y_pred_1)),
        "RMSE": np.sqrt(np.mean((y_test - y_pred_1) ** 2))
    }


#### Tabela com os resultados

Uma vez que temos o pandas, podemos utilizar este para apresentar a informação em formato de tabela.


TODO: Agora é preciso analisar os resultados e talvez produzir algum gráfico ou análise estatistica.

In [40]:
resultado = pd.DataFrame(resultados)

resultado.loc[["MAE", "RMSE"]]


,Genero,Historico_obesidade_familiar,FCCAC,FCV,NRP,CCER,Fumador,CA,MCC,FAF,TUDE,CBA,TRANS
MAE,0.174157,0.151583,0.169636,0.169275,0.174196,0.159079,0.174732,0.173374,0.169078,0.171724,0.173562,0.172866,0.174713
RMSE,0.211202,0.187125,0.204626,0.204894,0.211575,0.195568,0.211966,0.210912,0.206537,0.208206,0.210597,0.205701,0.211299


### 4.1.7

#### a) Utilizar Regressão Linear Múltipla

Target = IMC
Preditores = ?

Ver TP5


Holdout



In [41]:
y = dados_trabalho.IMC
X = dados_trabalho[lista_preditores].to_numpy() 


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, 
                                                    random_state = 100)

TODO: Imprimir a equação corretamente
TODO: Rever o código é cópia do TP5 

In [43]:
## Initialize algorithm
mlr = LinearRegression()

## Fit the data
mlr.fit(X_train,y_train)

print("Eq. da reta: y=", mlr.intercept_, "+", mlr.coef_, "x")

Eq. da reta: y= -0.11544856729411695 + [-0.00906778  0.20667286  0.06552097  0.17919736  0.00125902  0.07085282
  0.03465172  0.04235113 -0.067962   -0.09216686 -0.07044286  0.0479142
  0.0120811 ] x


In [44]:
#Prediction of test set
y_pred_mlr= mlr.predict(X_test)
#Predicted values
#print("Prediction for test set: {}".format(y_pred_mlr))

In [45]:
#Actual value and the predicted value
mlr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_pred_mlr})
mlr_diff.head()

,Actual value,Predicted value
785,0.324503,0.475563
1838,0.918202,0.603163
2083,0.737101,0.638463
1105,0.427048,0.398271
361,0.525597,0.444019


In [46]:
#Model Evaluation
from sklearn import metrics

meanAbErr = metrics.mean_absolute_error(y_test, y_pred_mlr)
meanSqErr = metrics.mean_squared_error(y_test, y_pred_mlr)

rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y_test, y_pred_mlr))

print('R squared: {:.2f}'.format(mlr.score(X,y)*100))
print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

R squared: 44.17
Mean Absolute Error: 0.126191413705803
Mean Square Error: 0.024166375606053066
Root Mean Square Error: 0.1554553813994648


#### b) DecisionTreeRegressor

Vamos usar os mesmos sets de treino e de teste.

TODO: Rever o código é cópia do TP5, ver TP7 tem uns testes que talvez sejam necessários.

In [ ]:
#tree = DecisionTreeRegressor(random_state=42)
tree = DecisionTreeRegressor(random_state=42, max_depth=6, min_samples_split=3)# Restrição de alguns parâmetros da árvore
model = tree.fit(X_train, y_train)

y_pred = tree.predict(X_train)
y_pred1 = tree.predict(X_test)



In [ ]:
MAE1 = metrics.mean_absolute_error(y_train, y_pred)
MAE2 = metrics.mean_absolute_error(y_test, y_pred1)
MAE3 = np.mean(abs(y_test-y_pred1)) #efetuando os cálculos

print("MAE on training set: {:.3f}".format(MAE1))
print("MAE on test set: {:.3f}".format(MAE2))
print("MAE on test set: {:.3f}".format(MAE3)) #efetuando os cálculos

RMSE = np.sqrt(np.mean((y_test-y_pred1)**2))
print("RMSE: {:.4f}".format(RMSE))

Visualização da Regression Tree

In [ ]:
# plot tree
from sklearn import tree

tree.plot_tree(model, 
          feature_names = list(df.columns), 
          class_names = list(df['sales']),  
          filled=True,                    
          fontsize=6);

In [ ]:
## the model:
from sklearn.tree import export_text
print(export_text(model,
                  show_weights=True))

#### c) MLPRegressor

TODO: Rever código é cópia do TP7

In [ ]:
from sklearn.neural_network import MLPRegressor

nn = MLPRegressor(hidden_layer_sizes=Nhidden,
                                          activation='tanh',
                                          solver='lbfgs', max_iter=1000, learning_rate_init=0.001)

In [ ]:
nn.fit(x_train, y_train)

In [ ]:
# Make prediction
pred = nn.predict(x_test)
#
# Calculate accuracy and error metrics
#
test_set_rsquared = nn.score(x_test, y_test)
test_set_rmse = np.sqrt(mean_squared_error(y_test, pred))
#
# Print R_squared and RMSE value
#
print('R_squared value: ', test_set_rsquared)
print('RMSE: ', test_set_rmse)

In [ ]:
print("weights between input and first hidden layer:")
print(nn.coefs_[0])
print("\nweights between first hidden and second hidden layer:")
print(nn.coefs_[1])

### 4.1.8

TODO Realizar os cálculos do MAE e RMSE do 4.1.7 colunas num DataFrame e imprimir

In [ ]:
comparar_417 = pd.DataFrame()
comparar_417

### 4.1.9

TODO: Estudo estatistico (ttest_ind) ? para os 2 modelos que apresentem melhores resultados.